<a href="https://colab.research.google.com/github/Struth-Rourke/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/Assignment_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [6]:
### YOUR CODE STARTS HERE

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-03-16 18:51:00--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.1’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2020-03-16 18:51:00 (286 KB/s) - ‘house-votes-84.data.1’ saved [18171/18171]



In [42]:
import pandas as pd
import numpy as np
from scipy import stats


names = ['party','handicapped-infants','water-project',
         'budget','physician-fee-freeze', 'el-salvador-aid',
         'religious-groups','anti-satellite-ban',
         'aid-to-contras','mx-missile','immigration',
         'synfuels', 'education', 'right-to-sue','crime','duty-free','south-africa']

df = pd.read_csv('house-votes-84.data', names = names)
df = df.replace({'y':1, 'n':0, '?':np.NaN})

print(df.shape)
print('.....')

print(df['party'].value_counts())
print('.....')

df.head()

(435, 17)
.....
democrat      267
republican    168
Name: party, dtype: int64
.....


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
rep = df[df['party'] == 'republican']
dem = df[df['party'] == 'democrat']

In [51]:
### Democrats support MORE than Republicans -- p-value < .01

print(stats.ttest_ind(dem['aid-to-contras'], rep['aid-to-contras'], nan_policy = 'omit'))

print('Democrats: ' + str(dem['aid-to-contras'].mean()))
print('Republicans: ' + str(rep['aid-to-contras'].mean()))

Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)
Democrats: 0.8288973384030418
Republicans: 0.15286624203821655


In [52]:
### Republicans support MORE than Democrats -- p-value < .01

print(stats.ttest_ind(dem['el-salvador-aid'], rep['el-salvador-aid'], nan_policy = 'omit'))

print('Democrats: ' + str(dem['el-salvador-aid'].mean()))
print('Republicans: ' + str(rep['el-salvador-aid'].mean()))

Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68)
Democrats: 0.21568627450980393
Republicans: 0.9515151515151515


In [53]:
### Republicans AND Democrats support -- p-value > .1

print(stats.ttest_ind(dem['water-project'], rep['water-project'], nan_policy = 'omit'))

print('Democrats: ' + str(dem['water-project'].mean()))
print('Republicans: ' + str(rep['water-project'].mean()))

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)
Democrats: 0.502092050209205
Republicans: 0.5067567567567568


## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables

In [165]:
def ind_ttest(x, y):
  print(stats.ttest_ind(x, y, nan_policy = 'omit'))
  print('Democrats: ' + str(x.mean()))
  print('Republicans: ' + str(y.mean()))

ind_ttest(dem['crime'], rep['crime'])

Ttest_indResult(statistic=-16.342085656197696, pvalue=9.952342705606092e-47)
Democrats: 0.35019455252918286
Republicans: 0.9813664596273292


In [0]:
def t_test(x, y):
  t_val, p_val = stats.ttest_ind(x,y,nan_policy='omit')
  mean_diff = x.mean() - y.mean()
  return [t_val, p_val, mean_diff]

In [166]:
t_test(rep['crime'], dem['crime'])

[16.342085656197696, 9.952342705606092e-47, 0.6311719070981463]

In [167]:
issues = df.columns.tolist()
issues.remove('party')
issues

['handicapped-infants',
 'water-project',
 'budget',
 'physician-fee-freeze',
 'el-salvador-aid',
 'religious-groups',
 'anti-satellite-ban',
 'aid-to-contras',
 'mx-missile',
 'immigration',
 'synfuels',
 'education',
 'right-to-sue',
 'crime',
 'duty-free',
 'south-africa']

In [0]:
stuff = {}
for issue in issues:
  stuff[issue] = t_test(rep[issue], dem[issue])

In [168]:
stuff

{'aid-to-contras': [-18.052093200819733,
  2.82471841372357e-54,
  -0.6760310963648253],
 'anti-satellite-ban': [-12.526187929077842,
  8.521033017443867e-31,
  -0.5314600314600315],
 'budget': [-23.21277691701378, 2.0703402795404463e-77, -0.7543151969981238],
 'crime': [16.342085656197696, 9.952342705606092e-47, 0.6311719070981463],
 'duty-free': [-12.853146132542978,
  5.997697174347365e-32,
  -0.5477066094595975],
 'education': [20.500685724563073, 1.8834203990450192e-64, 0.7263894286824718],
 'el-salvador-aid': [21.13669261173219,
  5.600520111729011e-68,
  0.7358288770053476],
 'handicapped-infants': [-9.205264294809222,
  1.613440327937243e-18,
  -0.41677237491190977],
 'immigration': [1.7359117329695164, 0.08330248490425066, 0.08609286784191733],
 'mx-missile': [-16.437503268542994,
  5.03079265310811e-47,
  -0.6429130009775171],
 'physician-fee-freeze': [49.36708157301406,
  1.994262314074344e-177,
  0.9338247338247339],
 'religious-groups': [9.737575825219457,
  2.393672252059

2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!

In [69]:
import pandas as pd
import numpy as np
from scipy import stats

## Define two random distributions

# Sample size
n = 20
# A dist -- Gaussian(Normal), mean = 2, variance = 1
a = np.random.randn(n) + 2
# B dist -- Gaussian(Normal), mean = 0, variance = 1
b = np.random.randn(n)


print(a.mean())
print(b.mean())

2.1493763929102063
-0.0865006132437616


In [72]:
## Calculate Variance and Standard Deviation

# Calculate Variance
# For unbiased likelihood we have to divide the var by n-1, so we set the degrees of freedom parameter to 1 (ddof = 1)
var_a = a.var(ddof = 1)
print('Variance of A: ' + str(var_a))

var_b = b.var(ddof = 1)
print('Variance of B: ' + str(var_b))

# Standard Deviation
s = np.sqrt((var_a + var_b)/2)
print('Standard Deviation between A and B: ' + str(s))

Variance of A: 0.9229372363589206
Variance of B: 1.5642316550004147
Standard Deviation between A and B: 1.1151611747544243


In [74]:
## Calculate the T-Test
t = (a.mean() - b.mean())/(s * np.sqrt(2/n))
t

6.340306735483214

In [80]:
## Compare with Critical T Value

# Degrees of freedom
dof = 2*n - 2

# P-value after comparison with the t
p = 1 - stats.t.cdf(t, df = dof)

# Print t-test and p-value
print("t = " + str(t))
print("p = " + str(2*p))

t = 6.340306735483214
p = 1.9451488597610478e-07


In [81]:
# Checking with scipy to see what values are generated

t2, p2 = stats.ttest_ind(a,b)
print("t = " + str(t2))
print("p = " + str(p2))

t = 6.340306735483214
p = 1.9451488590706386e-07
